## Autoregression Model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
AAPL = pd.read_csv('AAPL_New.csv')
AAPL.set_index('Date')
AAPL['CloseYesterday'] = AAPL['Close'].shift(-1)
AAPL['Open2mmorow'] = AAPL['Open'].shift(-1)
AAPL['Close2dayBefore'] = AAPL['Close'].shift(-2)
# AAPL = AAPL.fillna(method='ffill')
AAPL.tail()
# AAPL = AAPL.dropna(0)

,Date,High,Low,Open,Close,Volume,Adj Close,CloseYesterday,Open2mmorow,Close2dayBefore
983,11/27/2019,267.980011,265.309998,265.579987,267.839996,16308900,267.839996,267.250000,266.600006,264.160004
984,11/29/2019,268.000000,265.899994,266.600006,267.250000,11654400,267.250000,264.160004,267.269989,259.450012
985,12/2/2019,268.250000,263.450012,267.269989,264.160004,23621800,264.160004,259.450012,258.309998,261.739990
986,12/3/2019,259.529999,256.290008,258.309998,259.450012,28607600,259.450012,261.739990,261.070007,NaN
987,12/4/2019,263.309998,260.679993,261.070007,261.739990,15874176,261.739990,NaN,NaN,NaN


In [3]:
AAPL.tail(10)

,Date,High,Low,Open,Close,Volume,Adj Close,CloseYesterday,Open2mmorow,Close2dayBefore
978,11/20/2019,266.079987,260.399994,265.540008,263.190002,26558600,263.190002,262.010010,263.690002,261.779999
979,11/21/2019,264.010010,261.179993,263.690002,262.010010,30348800,262.010010,261.779999,262.589996,266.369995
980,11/22/2019,263.179993,260.839996,262.589996,261.779999,16331300,261.779999,266.369995,262.709992,264.290008
981,11/25/2019,266.440002,262.519989,262.709992,266.369995,21005100,266.369995,264.290008,266.940002,267.839996
982,11/26/2019,267.160004,262.500000,266.940002,264.290008,26301900,264.290008,267.839996,265.579987,267.250000
983,11/27/2019,267.980011,265.309998,265.579987,267.839996,16308900,267.839996,267.250000,266.600006,264.160004
984,11/29/2019,268.000000,265.899994,266.600006,267.250000,11654400,267.250000,264.160004,267.269989,259.450012
985,12/2/2019,268.250000,263.450012,267.269989,264.160004,23621800,264.160004,259.450012,258.309998,261.739990
986,12/3/2019,259.529999,256.290008,258.309998,259.450012,28607600,259.450012,261.739990,261.070007,NaN
987,12/4/2019,263.309998,260.679993,261.070007,261.739990,15874176,261.739990,NaN,NaN,NaN


In [4]:
def AR_bootstrap():
    # total num days
    N = AAPL.shape[0] 
    
    # total num bootstrap
    num_boot = 1
    
    # start day 
    T= 250 
    
    # training period window
    window = 200 
    
    Y =AAPL['Open2mmorow'][-(N-(T+1)):].values
    X = AAPL[['Close','CloseYesterday','Close2dayBefore']][-(N-(T+1)):].values
    T = X.shape[0]
#     print('T is: ')
#     print(T)
    X = np.column_stack([np.ones((T,1)),X])
    N = X.shape
    invXX = np.linalg.inv(X.transpose()@X)
    
    beta_hat = invXX@X.transpose()@Y
    #print(beta_hat.shape)
    y_hat = X@beta_hat+beta_hat[0]
    
    'Residuals from OLS'
    residuals = Y - y_hat
    'Variance of Residuals'
    sigma2 = (1/T)*residuals.transpose()@residuals
    'standard deviation of Y or residuals'
    sigma = np.sqrt(sigma2)
    'variance-covariance matrix of beta_hat'
    varcov_beta_hat = (sigma2)*invXX
    std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

    'Calculate R-square'
    R_square = 1 - (residuals.transpose()@residuals)/(T*np.var(Y))
    adj_R_square = 1-(1-R_square)*(T-1)/(T-N[1])

    'Test Each Coefficient: beta_i'
    'Null Hypothesis: beta_i = 0'
    t_stat = (beta_hat.transpose()-0)/std_beta_hat

    p_val = 1-ss.norm.cdf(t_stat)
    rmse_ar1 = np.sqrt(np.mean((Y- y_hat)**2))
    return y_hat,rmse_ar1


In [5]:
y,rmse=AR_bootstrap()
print(rmse)

nan


C:\Users\kaila\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\kaila\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\kaila\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\scipy\stats\_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


In [6]:
len(y)

737

In [7]:
AAPL.Date.iloc[-200:]

788     2/21/2019
789     2/22/2019
790     2/25/2019
791     2/26/2019
792     2/27/2019
          ...    
983    11/27/2019
984    11/29/2019
985     12/2/2019
986     12/3/2019
987     12/4/2019
Name: Date, Length: 200, dtype: object

In [8]:
AutoReg_predict = pd.DataFrame()
AutoReg_predict['Date'] = AAPL.Date.iloc[-200:]
AutoReg_predict['Predict_AutoReg'] = y[-200:]

In [9]:
AutoReg_predict.reset_index()
AutoReg_predict

,Date,Predict_AutoReg
788,2/21/2019,NaN
789,2/22/2019,NaN
790,2/25/2019,NaN
791,2/26/2019,NaN
792,2/27/2019,NaN
...,...,...
983,11/27/2019,NaN
984,11/29/2019,NaN
985,12/2/2019,NaN
986,12/3/2019,NaN


In [10]:
AutoReg_predict.to_csv('AutoReg_prediction.csv')